In [ ]:
import requests
import os
import pandas as pd
import numpy as np
import sys

#################################################################################EXTRAÇÂO DE DADOS#############################################################################

from google.colab import drive
drive.mount('/content/drive')


print("Menu Inicial\n")
valor = input("Digite:\n 1 para gerar dados de exportação de 2025\n 2 para gerar dados de 2024\n 3 para gerar dados de 2024 e 2025\n ")

if valor not in ['1', '2', '3']:
    sys.exit("❌ Entrada inválida. Encerrando o programa.")

# Diretório onde os arquivos serão salvos (pasta local do Google Drive)
pasta_destino = r'/content/drive/MyDrive/BECOMEX/entradas'
os.makedirs(pasta_destino, exist_ok=True)


pasta_saida = '/content/drive/MyDrive/BECOMEX/saidas'
os.makedirs(pasta_saida, exist_ok=True)  # Cria a pasta se não existir


# URLs e nomes dos arquivos desejados
arquivos = {
    "EXP_2024.csv": "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_2024.csv",
    "EXP_2025.csv": "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/EXP_2025.csv",
    "NCM.csv": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM.csv",
    "NCM_SH.csv": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_SH.csv",
    "NCM_ISIC.csv": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_ISIC.csv",
    "NCM_CGCE.csv": "https://balanca.economia.gov.br/balanca/bd/tabelas/NCM_CGCE.csv"
}

# Faz o download de cada arquivo
for nome_arquivo, url in arquivos.items():
    caminho_completo = os.path.join(pasta_destino, nome_arquivo)

    print(f"⬇ Baixando {nome_arquivo}...")
    try:
        resposta = requests.get(url, timeout=60)
        print(f"Status {resposta.status_code} para {nome_arquivo}")
        resposta.raise_for_status()
        with open(caminho_completo, 'wb') as f:
            f.write(resposta.content)
        print(f"✔ Salvo em: {caminho_completo}")
        print("Arquivo existe mesmo?", os.path.exists(caminho_completo))
    except Exception as e:
        print(f"❌ Erro ao baixar {nome_arquivo}: {e}")


#importando os caminho dos arquivos
caminho_ncm = f'{pasta_destino}/NCM.csv'
caminho_isic = f'{pasta_destino}/NCM_ISIC.csv'
caminho_exp2025 = f'{pasta_destino}/EXP_2025.csv'
caminho_sh = f'{pasta_destino}/NCM_SH.csv'
caminho_cgce = f'{pasta_destino}/NCM_CGCE.csv'
caminho_exp2024 = f'{pasta_destino}/EXP_2024.csv'

#definindo dataframes em python com PANDAS
df_ncm = pd.read_csv(caminho_ncm, sep=';', encoding='latin-1')
df_isic = pd.read_csv(caminho_isic, sep=';', encoding='latin-1')
df_exp2024 = pd.read_csv(caminho_exp2024, sep=';',encoding = 'latin-1',quotechar='"')
df_exp2025 = pd.read_csv(caminho_exp2025, sep=';', encoding='latin-1',quotechar='"')
df_sh = pd.read_csv(caminho_sh, sep=';', encoding='latin-1', on_bad_lines='skip')
df_cgce = pd.read_csv(caminho_cgce,  sep=';', encoding='latin-1')


#Verificando os tipos de dados
def verificar_tipos_de_dados():
    print("INFORMAÇÔES DOS TIPOS DE DADOS DA TABELA ISIC")
    print(df_isic.info())
    print("\n\n")
    print("INFORMAÇÔES DOS TIPOS DE DADOS DA TABELA NCM")
    print(df_ncm.info())
    print("\n\n")
    print("INFORMAÇÔES DOS TIPOS DE DADOS DA TABELA EXPORTAÇÃO 2025")
    print(df_exp2025.info())
    print("\n\n")
    print("INFORMAÇÔES DOS TIPOS DE DADOS DA TABELA SH")
    print(df_sh.info())
    print("\n\n")
    print("INFORMAÇÔES DOS TIPOS DE DADOS DA TABELA CGCE")
    print(df_cgce.info())
    print("\n\n")
    print("INFORMAÇÔES DOS TIPOS DE DADOS DA TABELA EXPPORTAÇÃO 2024")
    print(df_exp2024.info())
    print("\n\n")


#Verificando se existem celulas vazia
def verificar_celulas_vazias():
    print("\n\n")
    ver = {
        'df_ncm': df_ncm,
        'df_isic': df_isic,
        'df_exp2025': df_exp2025,
        'df_sh': df_sh,
        'df_cgce': df_cgce,
        'df_exp2024': df_exp2024
    }
#ver.items() retorna os dataframes do dicionario "ver"
    for nome, df in ver.items():
        print(f'\nDataFrame {nome}:')
#Verificando se há qualquer valor nulo no DataFrame
        if df.isna().any().any():
            print("⚠️ Existem células vazias conforme a tabela abaixo:")
            print(df.isna().sum())
        else:
            print("✅ Não existem células vazias.")

verificar_tipos_de_dados()
verificar_celulas_vazias()


colunas_para_remover = ['CO_PPE', 'CO_PPI', 'CO_FAT_AGREG', 'CO_SIIT', 'CO_EXP_SUBSET']
df_ncm = df_ncm.drop(columns=colunas_para_remover)




################################################################################### TRANSFORMANDO DADOS PT1 #######################################################################################
# Será relacionado as NCMS da industria da transformação que pertencem ao agro


def selecao_colunas_e_limpeza(df_sh, df_isic, df_cgce, df_ncm):

    df_sh_editada = df_sh[['CO_SH6', 'NO_SH6_POR', 'CO_SH4','NO_SH4_POR', 'CO_SH2', 'NO_SH2_POR','CO_NCM_SECROM', 'NO_SEC_POR']].copy()
    df_isic_editada = df_isic[['CO_ISIC_CLASSE', 'NO_ISIC_CLASSE', 'CO_ISIC_GRUPO', 'NO_ISIC_GRUPO',
                           'CO_ISIC_DIVISAO', 'NO_ISIC_DIVISAO', 'CO_ISIC_SECAO', 'NO_ISIC_SECAO']].copy()
    df_cgce_editada = df_cgce[['CO_CGCE_N3', 'NO_CGCE_N3',
       'CO_CGCE_N2', 'NO_CGCE_N2', 'CO_CGCE_N1', 'NO_CGCE_N1']].copy()
    df_ncm_editada = df_ncm[['CO_NCM', 'NO_NCM_POR', 'CO_SH6', 'CO_CGCE_N3', 'CO_ISIC_CLASSE' ]].copy()
    return df_sh_editada, df_isic_editada, df_cgce_editada, df_ncm_editada



df_cgce_editada['CO_CGCE_N3'] = df_cgce_editada['CO_CGCE_N3'].astype(str).str.strip()
    df_isic_editada['CO_ISIC_SECAO'] = df_isic_editada['CO_ISIC_SECAO'].astype(str).str.strip()
    for df in [df_sh_editada, df_isic_editada, df_cgce_editada, df_ncm_editada]:
        df.columns = df.columns.str.strip()

def gerar_df_de_interesse(df_ncm_editada, df_sh_agroeindustria, df_isic_AeC, df_cgce_Natura_e_Industria):
    """
    Realiza os left joins entre df_ncm_editada, df_sh_agroeindustria, df_isic_AeC e df_cgce_Natura_e_Industria
    seguindo a mesma lógica aplicada anteriormente, retornando apenas registros completos (sem valores nulos).
    """

    # Removendo espaços extras dos nomes das colunas
    df_ncm_editada.columns = df_ncm_editada.columns.str.strip()
    df_sh_agroeindustria.columns = df_sh_agroeindustria.columns.str.strip()
    df_isic_AeC.columns = df_isic_AeC.columns.str.strip()
    df_cgce_Natura_e_Industria.columns = df_cgce_Natura_e_Industria.columns.str.strip()

    # Garantindo os tipos corretos e padronizando os códigos como string sem espaços
    df_ncm_editada = df_ncm_editada.astype({'CO_ISIC_CLASSE': 'object', 'CO_CGCE_N3': 'object'})
    df_ncm_editada['CO_ISIC_CLASSE'] = df_ncm_editada['CO_ISIC_CLASSE'].astype(str).str.strip()
    df_ncm_editada['CO_CGCE_N3'] = df_ncm_editada['CO_CGCE_N3'].astype(str).str.strip()

    df_isic_AeC = df_isic_AeC.astype({'CO_ISIC_CLASSE': 'object'})
    df_isic_AeC['CO_ISIC_CLASSE'] = df_isic_AeC['CO_ISIC_CLASSE'].astype(str).str.strip()

    df_cgce_Natura_e_Industria = df_cgce_Natura_e_Industria.astype({'CO_CGCE_N3': 'object'})
    df_cgce_Natura_e_Industria['CO_CGCE_N3'] = df_cgce_Natura_e_Industria['CO_CGCE_N3'].astype(str).str.strip()

    # Primeiro left join entre NCM e SH
    df_juncao_NCMxSH = df_ncm_editada.merge(df_sh_agroeindustria, on='CO_SH6', how='left')

    # Left join com ISIC
    df_juncao_NCMxSHxISIC = df_juncao_NCMxSH.merge(df_isic_AeC, on='CO_ISIC_CLASSE', how='left')

    # Left join com CGCE
    df_juncao_NCMxSHxISICxCGCE = df_juncao_NCMxSHxISIC.merge(df_cgce_Natura_e_Industria, on='CO_CGCE_N3', how='left')


    df_NCM_SH_CGEC_ISIC_de_interesse = df_juncao_NCMxSHxISICxCGCE.copy()


    return df_NCM_SH_CGEC_ISIC_de_interesse



"-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"

#leftjoin com o Arquivo de exportação 2025

def join_to_EXP(df_NCM_SH_CGEC_ISIC_de_interesse, df_exp2025, df_exp2024):
    """
    Função que realiza a junção entre os DataFrames NCM, SH, ISIC, e os dados de exportação para os anos 2024 e 2025.

    Parâmetros:
    - df_NCM_SH_CGEC_ISIC_de_interesse: DataFrame com os dados NCM, SH, ISIC e CGCE já processados.
    - df_exp2025: DataFrame com os dados de exportação para o ano 2025.
    - df_exp2024: DataFrame com os dados de exportação para o ano 2024.

    Retorna:
    - DataFrame concatenado com os dados de exportação para os anos 2024 e 2025.
    """

    # Garantindo que 'CO_NCM' seja tratado como string e sem espaços extras
    df_NCM_SH_CGEC_ISIC_de_interesse = df_NCM_SH_CGEC_ISIC_de_interesse.astype({'CO_NCM': 'object'})
    df_NCM_SH_CGEC_ISIC_de_interesse['CO_NCM'] = df_NCM_SH_CGEC_ISIC_de_interesse['CO_NCM'].astype(str).str.strip()

    # Processando os DataFrames de exportação (2025 e 2024)
    df_exp2025 = df_exp2025.astype({'CO_NCM': 'object'})
    df_exp2025['CO_NCM'] = df_exp2025['CO_NCM'].astype(str).str.strip()

    df_exp2024 = df_exp2024.astype({'CO_NCM': 'object'})
    df_exp2024['CO_NCM'] = df_exp2024['CO_NCM'].astype(str).str.strip()

    concat = pd.concat([df_exp2024, df_exp2025])
    print(df_NCM_SH_CGEC_ISIC_de_interesse['CO_NCM'].nunique())
    print(df_NCM_SH_CGEC_ISIC_de_interesse.shape[0])

    df_NCM_SH_CGEC_ISIC_de_interesse['CO_SH6'] = pd.to_numeric(df_NCM_SH_CGEC_ISIC_de_interesse['CO_SH6'], errors='coerce')

    # Realizando o left join com o ano de 2024
    df_final = df_NCM_SH_CGEC_ISIC_de_interesse.merge(concat, on='CO_NCM', how='right')


    # Selecionando as colunas de interesse para ambos os DataFrames de 2024 e 2025
    colunas_interesse = ['CO_NCM', 'NO_NCM_POR', 'CO_SH6', 'CO_CGCE_N3', 'CO_ISIC_CLASSE', 'NO_SH6_POR',
                     'CO_SH4', 'NO_SH4_POR', 'CO_SH2', 'NO_SH2_POR', 'CO_NCM_SECROM', 'NO_SEC_POR',
                     'NO_ISIC_CLASSE', 'CO_ISIC_GRUPO', 'NO_ISIC_GRUPO', 'CO_ISIC_DIVISAO', 'NO_ISIC_DIVISAO',
                     'CO_ISIC_SECAO', 'NO_ISIC_SECAO', 'NO_CGCE_N3', 'CO_CGCE_N2', 'NO_CGCE_N2', 'CO_CGCE_N1',
                     'NO_CGCE_N1', 'CO_ANO', 'CO_MES', 'CO_UNID', 'CO_PAIS', 'SG_UF_NCM', 'CO_VIA', 'CO_URF',
                     'QT_ESTAT', 'KG_LIQUIDO', 'VL_FOB','Categoria']

    df_final = df_final[colunas_interesse].copy()

    print(df_final.isna().sum())
    # Concatenando os DataFrames de 2024 e 2025
    return df_final




def gerar_arquivos(df_NCM_SH_CGEC_ISIC_de_interesse, df_exp2025, df_exp2024, valor, nome_arquivo):
    exp2024, exp2025 = join_to_EXP(df_NCM_SH_CGEC_ISIC_de_interesse, df_exp2025, df_exp2024)

    # Geração dos arquivos em CSV

    if valor == '1':
        nome_arquivo_resultado = os.path.join(pasta_saida, f'EXP({nome_arquivo})2025.csv')
        exp2025.to_csv(nome_arquivo_resultado, index=False, sep=';', encoding='utf-8-sig')

    elif valor == '2':
        nome_arquivo_resultado = os.path.join(pasta_saida, f'EXP({nome_arquivo})2024.csv')
        exp2024.to_csv(nome_arquivo_resultado, index=False, sep=';', encoding='utf-8-sig')

    elif valor == '3':
        nome_arquivo_resultado_2024 = os.path.join(pasta_saida, f'EXP({nome_arquivo})2024.csv')
        nome_arquivo_resultado_2025 = os.path.join(pasta_saida, f'EXP({nome_arquivo})2025.csv')
        exp2024.to_csv(nome_arquivo_resultado_2024, index=False, sep=';', encoding='utf-8-sig')
        exp2025.to_csv(nome_arquivo_resultado_2025, index=False, sep=';', encoding='utf-8-sig')

    else:
        print("Número inválido")




df_sh_editada, df_isic_editada, df_cgce_editada, df_ncm_editada = selecao_colunas_e_limpeza(df_sh, df_isic, df_cgce, df_ncm)
nome_arquivo = 'ComexTratato'
df_NCM_SH_CGEC_ISIC_de_interesse = gerar_df_de_interesse(df_ncm_editada, df_sh_editada, df_isic_editada, df_cgce_editada)

df_NCM_SH_CGEC_ISIC_de_interesse['CO_NCM'] = df_NCM_SH_CGEC_ISIC_de_interesse['CO_NCM'].astype(int)
# Filtro correto para Agro
filtro_agro = df_NCM_SH_CGEC_ISIC_de_interesse['NO_ISIC_SECAO'] == 'Agropecuária'
df_NCM_SH_CGEC_ISIC_agro = df_NCM_SH_CGEC_ISIC_de_interesse[filtro_agro].copy()
df_NCM_SH_CGEC_ISIC_agro.loc[:, 'Categoria'] = 'Agropecuária'

# Filtro correto para Indústria (sem duplicar a lógica anterior)
filtro_agroindustria = ((df_NCM_SH_CGEC_ISIC_de_interesse['NO_ISIC_SECAO'] == 'Indústria de Transformação') & (df_NCM_SH_CGEC_ISIC_de_interesse['CO_SH6'] <= 240000))
df_NCM_SH_CGEC_ISIC_agroindustria = df_NCM_SH_CGEC_ISIC_de_interesse[filtro_agroindustria].copy()
df_NCM_SH_CGEC_ISIC_agroindustria.loc[:, 'Categoria'] = 'Agroindustria'

# Filtro correto para Indústria (sem duplicar a lógica anterior)
filtro_industria = ((df_NCM_SH_CGEC_ISIC_de_interesse['NO_ISIC_SECAO'] == 'Indústria de Transformação') & (df_NCM_SH_CGEC_ISIC_de_interesse['CO_SH6'] > 240000))
df_NCM_SH_CGEC_ISIC_industria = df_NCM_SH_CGEC_ISIC_de_interesse[filtro_industria].copy()
df_NCM_SH_CGEC_ISIC_industria.loc[:, 'Categoria'] = 'Indústria de Transformação'

# ❌ Aqui estava aplicando o filtro_agro por engano — CORRIGIDO:
filtro_extrativa = df_NCM_SH_CGEC_ISIC_de_interesse['NO_ISIC_SECAO'] == 'Indústria Extrativa'
df_NCM_SH_CGEC_ISIC_extrativa = df_NCM_SH_CGEC_ISIC_de_interesse[filtro_extrativa].copy()
df_NCM_SH_CGEC_ISIC_extrativa.loc[:, 'Categoria'] = 'Indústria Extrativa'

# ❌ Aqui também estava aplicando o filtro_agro — CORRIGIDO:
filtro_outros = df_NCM_SH_CGEC_ISIC_de_interesse['NO_ISIC_SECAO'] == 'Outros Produtos'
df_NCM_SH_CGEC_ISIC_outros = df_NCM_SH_CGEC_ISIC_de_interesse[filtro_outros].copy()
df_NCM_SH_CGEC_ISIC_outros.loc[:, 'Categoria'] = 'Outros Produtos'

#concat_de_todos
df_concatenado = pd.concat([
    df_NCM_SH_CGEC_ISIC_agro,
    df_NCM_SH_CGEC_ISIC_agroindustria,
    df_NCM_SH_CGEC_ISIC_industria,
    df_NCM_SH_CGEC_ISIC_extrativa,
    df_NCM_SH_CGEC_ISIC_outros
], ignore_index=True)


nome_arquivo_resultado = os.path.join(pasta_saida, 'df relacionado.csv')
df_concatenado.to_csv(nome_arquivo_resultado, index=False, sep=';', encoding='utf-8-sig')

df_final = join_to_EXP(df_concatenado, df_exp2025, df_exp2024)
caminho = os.path.join(pasta_saida, 'tabelafinal.csv')
df_final.to_csv(caminho, index=False, sep=';', encoding='utf-8-sig')

print(df_final.shape[0])
print(df_exp2024.shape[0] + df_exp2025.shape[0])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Menu Inicial

Digite:
 1 para gerar dados de exportação de 2025
 2 para gerar dados de 2024
 3 para gerar dados de 2024 e 2025
 3
⬇ Baixando EXP_2024.csv...
Status 200 para EXP_2024.csv
✔ Salvo em: /content/drive/MyDrive/BECOMEX/entradas/EXP_2024.csv
Arquivo existe mesmo? True
⬇ Baixando EXP_2025.csv...
Status 200 para EXP_2025.csv
✔ Salvo em: /content/drive/MyDrive/BECOMEX/entradas/EXP_2025.csv
Arquivo existe mesmo? True
⬇ Baixando NCM.csv...
Status 200 para NCM.csv
✔ Salvo em: /content/drive/MyDrive/BECOMEX/entradas/NCM.csv
Arquivo existe mesmo? True
⬇ Baixando NCM_SH.csv...
Status 200 para NCM_SH.csv
✔ Salvo em: /content/drive/MyDrive/BECOMEX/entradas/NCM_SH.csv
Arquivo existe mesmo? True
⬇ Baixando NCM_ISIC.csv...
Status 200 para NCM_ISIC.csv
✔ Salvo em: /content/drive/MyDrive/BECOMEX/entradas/NCM_ISIC.csv
Arquivo existe mesmo? True
⬇ Baixando NCM_CGCE.cs

In [ ]:
print(df_final.shape[0])
print(df_exp2024.shape[0] + df_exp2025.shape[0])

2137260
2137260


In [ ]:
df_final.duplicated().sum()

np.int64(0)